# Converting data to h3 at zoom 8

In [1]:
from cng.utils import *
from cng.h3 import *
from ibis import _
import os
from osgeo import gdal
from minio import Minio
import streamlit 
from datetime import timedelta

# Get signed URLs to access license-controlled layers
key = st.secrets["MINIO_KEY"]
secret = st.secrets["MINIO_SECRET"]
client = Minio("minio.carlboettiger.info", key, secret)

con = ibis.duckdb.connect(extensions = ["spatial", "h3"])
# endpoint = os.getenv("AWS_S3_ENDPOINT", "minio.carlboettiger.info")
duckdb_install_h3()

set_secrets(con)

In [ ]:
def h3_from_geom(con, name, cols, zoom = 8):
    """
    Computes hexes directly from geometry.
    """
    cols = ", ".join(cols) if isinstance(cols, list) else cols
    con.raw_sql(f'''
    CREATE OR REPLACE TEMP TABLE t2 AS
    SELECT {cols},
           h3_polygon_wkt_to_cells_string(ST_Force2D(dump.geom), {zoom}) AS h{zoom}
    FROM (
        SELECT {cols}, UNNEST(ST_Dump(geom)) AS dump
        FROM {name}
    )
    ''')
    
    con.sql(f'''
        SELECT {cols}, UNNEST(h{zoom}) AS h{zoom},
        ST_GeomFromText(h3_cell_to_boundary_wkt(UNNEST(h{zoom}))) AS geom
        FROM t2
    ''').to_parquet(f"{name}_h3_z{zoom}.parquet")

    client.fput_object(bucket_name = "shared-tpl",
               object_name = f"{name}_h3_z{zoom}.parquet",
               file_path = f"{name}_h3_z{zoom}.parquet") 
    return 

### TPL Conservation Almanac

In [ ]:
tpl = client.get_presigned_url(
    "GET",
    "shared-tpl",
    "tpl.parquet",
    expires=timedelta(hours=2),
)

cols = ['fid', 'TPL_ID', 'State', 'County', 'Municipality',
       'Site_Name', 'Reported_Acres', 'Close_Year', 'Close_Date', 'Owner_Name',
       'Owner_Type', 'Manager_Name', 'Manager_Type', 'Purchase_Type',
       'EasementHolder_Name', 'EasementHolder_Type', 'Public_Access_Type',
       'Purpose_Type', 'Duration_Type', 'Data_Provider', 'Data_Source',
       'Source_Date', 'Data_Aggregator', 'Comments', 'Amount', 'Program_ID',
       'Program_Name', 'Sponsor_ID', 'Sponsor_Name', 'Sponsor_Type']


tpl_table = (con.read_parquet(tpl)
             .mutate(geom = _.geom.convert("ESRI:102039", "EPSG:4326"))
            )

con.create_table('tpl', tpl_table, overwrite=True)
h3_from_geom(con, 'tpl', cols)

In [2]:
tpl = client.get_presigned_url(
    "GET",
    "shared-tpl",
    "tpl_h3_z8.parquet",
    expires=timedelta(hours=2),
)
con.read_parquet(tpl).head().execute()

,fid,TPL_ID,State,County,Municipality,Site_Name,Reported_Acres,Close_Year,Close_Date,Owner_Name,...,Data_Aggregator,Comments,Amount,Program_ID,Program_Name,Sponsor_ID,Sponsor_Name,Sponsor_Type,h8,geom
0,15,10205389,Alabama,Baldwin,None,BON SECOUR NATIONAL WILDLIFE REFUGE 19,24.00,2006,2006-12-29 00:00:00+00:00,None,...,TPL,No final policy or deed has been received from...,102150.0,5754,Land and Water Conservation Fund (LWCF),2356,U.S. Fish and Wildlife Service,FED,884450d991fffff,"POLYGON ((-87.76568 30.25687, -87.7609 30.2591..."
1,18,10205396,Alabama,Baldwin,None,BON SECOUR NATIONAL WILDLIFE REFUGE 18,144.00,2003,2003-10-15 00:00:00+00:00,None,...,TPL,Fund code: 8422BONX.,1185000.0,5754,Land and Water Conservation Fund (LWCF),2356,U.S. Fish and Wildlife Service,FED,884450c265fffff,"POLYGON ((-87.91775 30.23324, -87.91296 30.235..."
2,22,10205402,Alabama,Bibb,None,CAHABA RIVER NATIONAL WILDLIFE REFUGE 1,304.00,2002,2002-09-25 00:00:00+00:00,None,...,TPL,This tract was the lst tract acquired at this ...,829000.0,5754,Land and Water Conservation Fund (LWCF),2356,U.S. Fish and Wildlife Service,FED,8844e8b629fffff,"POLYGON ((-87.08426 33.05601, -87.07937 33.058..."
3,23,10205401,Alabama,Bibb,None,CAHABA RIVER NATIONAL WILDLIFE REFUGE 5,304.00,2003,2003-06-20 00:00:00+00:00,None,...,TPL,Cahaba River Act P.L. 106-331(114 Stat. 1304-1...,782000.0,5754,Land and Water Conservation Fund (LWCF),2356,U.S. Fish and Wildlife Service,FED,8844e8b62dfffff,"POLYGON ((-87.07463 33.05544, -87.06975 33.057..."
4,24,10205398,Alabama,Bibb,None,CAHABA RIVER NATIONAL WILDLIFE REFUGE 3,165.09,2002,2002-11-26 00:00:00+00:00,None,...,TPL,Cahaba River Act P.L. 106-331(114 Stat. 1304-1...,212500.0,5754,Land and Water Conservation Fund (LWCF),2356,U.S. Fish and Wildlife Service,FED,8844e8b623fffff,"POLYGON ((-87.08382 33.07068, -87.07893 33.072..."


### Landvote

In [ ]:
landvote = client.get_presigned_url(
    "GET",
    "shared-tpl",
    "landvote.csv",
    expires=timedelta(hours=2),
)

con.read_csv(landvote)